# Time to Train my first LLM
---



## Setup

In [ ]:
# install unsloth and pandas
!pip install unsloth
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.5/203.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 865.2/865.2 MB 244.9 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
# import the libs that our teacher told us to
from unsloth import FastLanguageModel
import torch

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
# set the max number of tokens that can be passed in per chunk
max_seq_length = 2048

# let unsloth select our data type for us
dtype = None

# enable 4-bit quantization
# less memory but slightly less accuracy its just some math stuff
load_in_4bit = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# find path to my data
!ls 'drive/MyDrive/intro to AI/Notebooks'

In [ ]:
import json
import pandas as pd
from datasets import Dataset

# connect to our data path
dataPath = 'drive/MyDrive/intro to AI/Notebooks/allMethodsCombined.json'

# load and convert to DataFrame
with open(dataPath, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)  # <- make it a DataFrame

# save the DataFrame to CSV
df.to_csv('allMethodsCombined.csv', index=False)

# convert to a hf dataset
dataset = Dataset.from_pandas(df)


In [ ]:
# make a 2 col dataset
df2col = df[['code', 'javadoc']]

dataset2col = Dataset.from_pandas(df)


## Model Selections:
- Qwen 2.5 coder (0.5b)
- Codellama (7b)
- Wizardcoder (33b)


In [ ]:
# models from hugging face
model1 = 'Qwen/Qwen2.5-Coder-0.5B-Instruct'
model2 = 'WizardLMTeam/WizardCoder-33B-V1.1'
model3 = 'codellama/CodeLlama-7b-hf'


## Fine tuning Qwen 2.5 Coder (0.5B)

### Setting up model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # qwen coder 0.5b
    model_name= model1,
    max_seq_length=max_seq_length,
    dtype = dtype,
    # give each model 4bit quantization or however you spell it
    load_in_4bit= True
)

### Establish How the model will learn

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

### Start Training (Can be changed)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset2col,
    dataset_text_field="code",
    args=TrainingArguments(
        output_dir="output",
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=2e-4,
        report_to="none",
    ),
)


In [ ]:
trainer.train()


## Upload to Hugging Face

In [ ]:
from huggingface_hub import login

# Log in first (you only need to do this once)
login()

# Push model
model.push_to_hub("Eyas1/fine_tuned_qwen_java_docs")
tokenizer.push_to_hub("Eyas1/fine_tuned_qwen_java_docs")


## Save the model to drive

In [ ]:
model.save_pretrained("drive/MyDrive/fine_tuned_qwen_java_docs")
tokenizer.save_pretrained("drive/MyDrive/fine_tuned_qwen_java_docs")
